Λαμπρόπουλος Κωνσταντίνος $$ $$
ΑΜ : 1115201800092

Imports

In [ ]:
import pandas as pd
import numpy as np
from PIL import Image
import random
import glob
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
pd.set_option("display.max_rows",None)
pd.set_option("display.max_columns",None)

Load Image

In [ ]:
def SetRange(set_number): #Set the aprropriate ranges for each set
    if set_number == 1: #not that the in range() function of python iterates over the [start,stop) range
        infimum,maximum = 1,8
    elif set_number == 2:
        infimum,maximum = 8,20
    elif set_number == 3:
        infimum,maximum = 20,32
    elif set_number == 4:
        infimum,maximum = 32,46
    elif set_number == 5:
        infimum,maximum = 46,65
    return (infimum,maximum)

In [ ]:
def LoadImages(path,set_number): #load an image set
    loadedimages = [] #empty lists to store our variables
    labels = []
    start,stop = SetRange(set_number) #get the range of each set
    for i in range(1,11): #we will get a number of images from all people
        for j in range(start,stop): #but in each set we take different pictures from all people
            if i < 10: #images of people from 1-9 have format like this
                if j < 10: #e.g person03_09.png so we need to add the 0s to the path
                    image_path = f"{path}/person0{i}_0{j}.png"
                else:
                    image_path = f"{path}/person0{i}_{j}.png" 
            else: #else if it is the 10nth person,we dont need a 0 in the beggining
                if j < 10:
                    image_path = f"{path}/person{i}_0{j}.png"
                else:
                   image_path = f"{path}/person{i}_{j}.png" 
            image = Image.open(image_path) #open the image from the path
            loadedimages.append(np.array(image).flatten()) #get a vector from the image
            labels.append(i) #and add the according label
    return loadedimages,labels #retun a tuple

Train Eigen Faces

In [ ]:
def TrainEigenFaces(train_data, train_labels, d):
    pca = PCA(n_components=d) #initialize PCA with the components we wnat
    data = pca.fit_transform(train_data) #project data to eigen space
    
    knn = KNeighborsClassifier(n_neighbors=1,p=2) #use Knn neighbours classifier
                                              #the classifier uses euclidiean distance metric when
                                              #variable p equals to 2
    knn.fit(data, train_labels) #fit the data

    return pca,knn,pca.components_ #return tuple

Test Eigen Faces

In [ ]:
def TestEigenFaces(pca, knn, test_data):
    data = pca.transform(test_data) #project our test data to eigen space
    predicted_labels = knn.predict(data) #predict the image we are using

    return predicted_labels #return predictions

In [ ]:
def ImageReconstrunction(eigenvectors,pca,image):
    vector = image.flatten() - pca.mean_
    coefficients = []
    for eigenvector in eigenvectors:
        coefficient  = np.dot(vector,eigenvector)
        coefficients.append(coefficient)
    np.array(coefficients)
    coefficients = np.reshape(coefficients,(len(pca.components_),1))
    eigenvectors = np.transpose(eigenvectors)
    vector = np.dot(eigenvectors,coefficients)
    recontructed_image = np.array(vector).reshape(50,50)
    plt.figure(figsize=(9,4))
    plt.subplot(3,8,1)
    plt.imshow(image.reshape(50,50),cmap='bone')
    plt.subplot(3,8,2)
    plt.imshow(recontructed_image,cmap='bone')
    plt.title(f'Image Reconstrunction for d={len(pca.components_)}')


Face Recognition

In [ ]:
def FaceRecognition():
    train_data,train_labels = LoadImages('./faces',1) #load our training data
    pca,knn,eigenvectors = TrainEigenFaces(train_data,train_labels,9) #perform face recognition with 9 vectors first
    print("Accuracy Scores for 9 vectors")
    rand = random.randint(1,5)
    for i in range (1,6):
        test_data,test_labels = LoadImages('./faces',i) #load test sets
        if i == rand:
            ImageReconstrunction(eigenvectors,pca,test_data[i])
        predicted_labels = TestEigenFaces(pca,knn,test_data) #test each set
        print(f"Set {i} : {accuracy_score(predicted_labels,test_labels)*100:.6f}%") #print accuracy of predicitons
    pca,knn,eigenvectors_2 = TrainEigenFaces(train_data,train_labels,30) #perform face recognition with 30 vectors
    print("Accuracy Score for 30 vectors")
    rand = random.randint(1,5)
    for i in range (1,6):
        test_data,test_labels = LoadImages('./faces',i) #load test sets
        if i == rand:
            ImageReconstrunction(eigenvectors_2,pca,test_data[i])
        predicted_labels = TestEigenFaces(pca,knn,test_data) #project set to eigen space and predict each image
        print(f"Set {i} : {accuracy_score(predicted_labels,test_labels)*100:.6f}%") #print accuracy
    return eigenvectors

Convert Eigen Vectors To Image

In [ ]:
def EigenVectorToImage(eigenvectors):
    plt.figure(figsize=(9,4))
    for i in range(1,10,1):
        plt.subplot(3,8,i)
        plt.imshow(eigenvectors[i-1].reshape(50,50),cmap='bone')
    plt.title(label="EigenVectors",loc='left')

Τα ιδιοδιανύσματα αυτά εκφράζουν τις πιο σημαντικές αλλαγές στο train set.Ειδικότερα
πιάνουν τις εναλλαγές στον φωτισμό και στις εκφράσεις του προσώπου

Image Reconstruction

In [ ]:
eigenvectors = FaceRecognition() #Call the function to showcase the results
EigenVectorToImage(eigenvectors)